# LangChain Basics for Email Agent
## Goal
Understand core langchain components needed for building an email agent 

## Topics
- LLM initialization
- Prompt templates
- Basic invocation

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
print("OPENAI_API_KEY exists:", bool(os.getenv("OPENAI_API_KEY")))

OPENAI_API_KEY exists: True


In [2]:
import sys
print(sys.version)

3.14.0 (v3.14.0:ebf955df7a8, Oct  7 2025, 08:20:14) [Clang 16.0.0 (clang-1600.0.26.6)]


In [3]:
import os

print("OPENAI_API_KEY exists:", bool(os.getenv("OPENAI_API_KEY")))


OPENAI_API_KEY exists: True


In [4]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')
result = llm.invoke("what is the capital of India")
print(result)

/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


content='The capital of India is New Delhi.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 13, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e413f45763', 'id': 'chatcmpl-CosbmQxOgCnbbVUvMTCAf7ubmeOqL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b3c3f-7359-7ac0-a770-9b72ee5e2be0-0' usage_metadata={'input_tokens': 13, 'output_tokens': 8, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [6]:
from langchain_core.prompts import PromptTemplate

In [7]:
email_prompt = PromptTemplate(
    input_variables=["sender", "subject", "body"], 
    template ="""
    You are an email assistant.

Sender: {sender}
Subject: {subject}

Email Body:
{body}

Task:
Write a short professional reply.
"""
)

In [8]:
formatted_prompt = email_prompt.format(
    sender = "HR Team",
    subject = "Interview Schedule",
    body = "We would like to schedule the interview next week. Please confirm availability."
)
print(formatted_prompt)


    You are an email assistant.

Sender: HR Team
Subject: Interview Schedule

Email Body:
We would like to schedule the interview next week. Please confirm availability.

Task:
Write a short professional reply.



In [9]:
response = llm.invoke(formatted_prompt)
print(response.content)


Subject: Re: Interview Schedule

Dear HR Team,

Thank you for considering me for the interview. I am available next week and can accommodate the schedule as needed. Please let me know the specific date and time that work best for you.

Looking forward to the interview.

Best regards,

[Your Name]


In [10]:
import sys
sys.path.append("../src")


In [11]:
from email_agent.prompts import EMAIL_REPLY_PROMPT
from email_agent.llm import get_llm

llm = get_llm()
formatted_prompt = EMAIL_REPLY_PROMPT.format(
    sender = "HR Team",
    subject="Interview Schedule",
    body="We would like to schedule your interview next week. Please confirm availability."
)
response = llm.invoke(formatted_prompt)
print(response.content)

Subject: Re: Interview Schedule

Dear HR Team,

Thank you for the opportunity. I am available for an interview next week and would appreciate any details regarding the date and time. 

I look forward to our conversation.

Best regards,

[Your Name]


In [12]:
from email_agent.prompts import EMAIL_CLASSIFICATION_PROMPT
classification_prompt = EMAIL_CLASSIFICATION_PROMPT.format(
    subject = "Interview Schedule",
    body = "We would like to schedule your interview next week. Please confirm availability."
)
classification = llm.invoke(classification_prompt)
print(classification.content)

request


In [13]:
from email_agent.router import route_email
category = classification.content.strip().lower()
action = route_email(category)

print("Category:", category)
print("Action:", action)


Category: request
Action: reply


In [14]:
import inspect 
from email_agent.router import route_email
print(inspect.getsource(route_email))

def route_email(category: str) -> str:
    """
    Decide what action to take based on email category.
    """
    category = category.strip().lower()

    if category == "request":
        return "reply"
    elif category == "urgent":
        return "escalate"
    elif category == "information":
        return "acknowledge"
    elif category == "spam":
        return "ignore"
    else:
        return "unknown"



In [15]:
pip install langgraph


Note: you may need to restart the kernel to use updated packages.


In [16]:
import langgraph
print(langgraph.__version__)

AttributeError: module 'langgraph' has no attribute '__version__'

In [17]:
import importlib.metadata

importlib.metadata.version("langgraph")


'1.0.4'